# Tunisian Horses


In [ ]:
import os
import pandas as pd
import re
from PIL import Image

# Set the path to the main directory containing the folders of images
main_dir = '../data/THoDBRL2015'
search_in_dir = 'Croped Images'

# Initialize a list to hold metadata
metadata = []

# Recursively walk through each folder and subfolder to collect metadata
for root, dirs, files in os.walk(main_dir):
    # 'root' is the current directory in the hierarchy
    # 'dirs' are the directories in 'root'
    # 'files' are the files in 'root'
    # basename = os.path.basename(root)
    if search_in_dir in root:
        for filename in files:
            if filename.endswith(('.png', '.jpg', '.jpeg')):  # Adjust extensions as needed
                file_path = os.path.join(root, filename)

                # Extract the folder name as the relative path from main_dir to the current folder
                relative_folder_path = os.path.relpath(root, main_dir)

                match_horse_id = re.search(r'\d+$', relative_folder_path)
                horse_id = match_horse_id.group() if match_horse_id else None  # Assign None if no match is found

                # Determine category based on filename content
                if 'Dhr' in filename: # Droit
                    camera_position_category = 'Right'
                elif 'Ghr' in filename: # Gauche
                    camera_position_category = 'Left'
                elif 'fhr' in filename: # front
                    camera_position_category = 'Front'
                else:
                    camera_position_category = 'Unknown'

                match_position = re.sub(r'\d', '', filename)
                # camera_position = match_position.group() if match_position else None

                with Image.open(main_dir + '/' + relative_folder_path + '/' + filename) as img:
                    width, height = img.size  # .size returns (width, height)

                # Determine orientation
                if width > height:
                    orientations = 'Landscape'
                elif height > width:
                    orientation = 'Portrait'
                else:
                    orientation = 'Square'


                # Add metadata: Folder path, filename, and full file path
                metadata.append({
                    'file_path': file_path,
                    'horse_id': horse_id,
                    'photo_id': re.search(r'\d+', filename).group(),
                    'folder': relative_folder_path,  # This will give the relative path of the folder structure
                    'filename': filename,
                    'format': filename.split('.')[-1],
                    'camera_position': camera_position_category,
                    'file_size': os.path.getsize(file_path),
                    'width': width,
                    'height': height,
                    'aspect_ratio': width / height,
                    'orientation': orientation,
                })

# Create DataFrame with specified dtypes
metadata_df = pd.DataFrame(metadata, dtype='object').astype({
    'file_path': 'string',
    'horse_id': 'int64',          # Assuming it's an integer
    'photo_id': 'int64',          # Assuming it's an integer
    'folder': 'string',
    'filename': 'string',
    'format': 'category',        # Categorical for file format
    'camera_position': 'category', # Categorical for camera position
    'file_size': 'int64',         # Integer for file size in bytes
    'width': 'int64',             # Integer for width
    'height': 'int64',             # Integer for height
    'aspect_ratio': 'float',
    'orientation': 'category',
})
# Convert metadata list to a DataFrame
# metadata_df = pd.DataFrame(metadata)
metadata_df

In [ ]:
metadata_df['horse_id'].unique()

In [ ]:
metadata_df.dtypes

In [ ]:
metadata_df['camera_position'].unique()

In [ ]:
metadata_df['aspect_ratio'].unique()

In [ ]:
metadata_df['orientation'].unique()

In [ ]:
grouped_horse_camera = metadata_df.groupby(['horse_id', 'camera_position'], observed=True)
grouped_horse_camera.size()


In [ ]:
grouped_horse_camera

In [ ]:
# Apply different aggregations for each column
agg_df = grouped_horse_camera.agg({
    'file_size': ['min', 'max'],
     # 'file_size': 'max'
    'aspect_ratio': 'max',
    # 'brightness': ['min', 'max']  # Min and max of brightness
})
print("Aggregated DataFrame with multiple functions:")
print(agg_df)

In [ ]:
grouped_horse = metadata_df.groupby(['horse_id'], observed=True)
agg_horse_df = grouped_horse.agg({
    'file_size': ['min', 'max', 'mean', 'std'],
})
agg_horse_df

In [ ]:
grouped_camera_position = metadata_df.groupby(['camera_position'], observed=True)
agg_camera_position_df = grouped_camera_position.agg({
    'file_size': ['min', 'max', 'mean', 'std'],
    'width': ['min', 'max', 'mean'],
    'height': ['min', 'max', 'mean'],
})
agg_camera_position_df

So, all the Front images are 160x380. The Left and Right images are 165x260.

In [ ]:
from matplotlib import pyplot as plt

# Retain only relevant columns for displaying images by horse ID
data = metadata_df[['horse_id', 'file_path']]


# Drop duplicates to get one image per horse
data_unique_horses = data.drop_duplicates(subset=['horse_id'])

# Set grid size to display 47 images (8 columns, 6 rows)
cols, rows = 8, 6

# Create a figure with a static grid of subplots
fig, axes = plt.subplots(rows, cols, figsize=(20, 15), dpi=100)  # Set DPI for higher resolution

# Flatten the axes array for easy iteration
axes = axes.flatten()

# Display each selected image
for i, (idx, row) in enumerate(data_unique_horses.iterrows()):
    img_path = row['file_path']

    # Check if the file path is valid before loading
    if os.path.exists(img_path):
        img = Image.open(img_path).convert("RGB")  # Ensure RGB mode
        axes[i].imshow(img, interpolation='nearest')  # Avoid interpolation
    else:
        # Display a placeholder text if image path is invalid
        axes[i].text(0.5, 0.5, "Image Not Found", ha='center', va='center', color="red")

    # Hide axes and set title as horse ID
    axes[i].axis('off')
    axes[i].set_title(f"Horse ID: {row['horse_id']}")

# Hide any remaining empty subplots
for j in range(i + 1, len(axes)):
    axes[j].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split


In [ ]:
import tensorflow.keras as keras

In [ ]:
from tensorflow.keras.utils import to_categorical


In [ ]:
import numpy as np
print(np.__version__)

In [ ]:


print(y_train.max())  # Max label value
num_classes = len(metadata_df['horse_id'].unique())
print(num_classes)  # Check number of classes

In [ ]:


# Prepare dataset
def load_images_and_labels(metadata_df):
    images = []
    labels = []
    
    for _, row in metadata_df.iterrows():
        if os.path.exists(row['file_path']):
            img = Image.open(row['file_path']).resize((128, 128))  # Scale to same size
            images.append(np.array(img))
            labels.append(row['horse_id'])  # use horse_id for the label
    
    images = np.array(images)
    labels = np.array(labels)
    
    return images, labels

# Load data
images, labels = load_images_and_labels(metadata_df)

# Split in train and test sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Normalise pixelvalues
X_train = X_train / 255.0
X_test = X_test / 255.0

# Transfer labels to category data
num_classes = len(metadata_df['horse_id'].unique())
#y_train = y_train - 1  # If needed then custumize labels (sometimes gives error)
#y_test = y_test - 1  # The same for test

# Convert labels to categorical data
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)



In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),  # Prevent overfitting
    Dense(num_classes, activation='softmax')  # Outputlayer
])

# Add layers step-by-step
model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(128, 128, 3)))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))  # To prevent overfitting

model.add(Dense(num_classes, activation='softmax'))  # Output layer

# Model summary
model.summary()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Modeloverview
model.summary()


In [ ]:
history = model.fit(
    X_train, y_train,
    epochs=10,
    batch_size=32,
    validation_data=(X_test, y_test)
)


In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")


In [ ]:
model.save('horse_identifier_cnn.h5')


In [ ]:
from tensorflow.keras.models import load_model

# load the model
model = load_model('horse_identifier_cnn.h5')

# Classify a new imagae
def classify_image(img_path):
    img = Image.open(img_path).resize((128, 128))
    img_array = np.array(img) / 255.0  # Normalise
    img_array = np.expand_dims(img_array, axis=0)  # add a batch dimension
    
    prediction = model.predict(img_array)
    predicted_class = np.argmax(prediction)
    return predicted_class

new_image_path = "Testfotos Paard.jpg" #random photo of horse from internet
predicted_horse = classify_image(new_image_path)
print(f"Predicted Horse ID: {predicted_horse}")



In [ ]:


# Evaluate Model on testset
test_loss, test_accuracy = model.evaluate(X_test, y_test)

# print the results
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

class_names = [str(i) for i in range(num_classes)]  

# predict the labels for testset
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)

# Generate the confusion matrix
cm = confusion_matrix(np.argmax(y_test, axis=1), y_pred_classes)

# Plot The confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted Class')
plt.ylabel('Real Class')
plt.title('Confusion Matrix')
plt.show()
